# Detecção de Cantos

Prof. Dr. Gustavo Resque  
MSc. Tiago Araújo

### Cantos

exemplo de imagem para cantos, região homogênea e borda (np.zeros e np.ones) (ver rotação e escala widget)  
exemplo com imagem_canto

### Moravec

fórmula  
explicar os elementos da fórmula (?)   
exemplo binário  
exemplo grayscale (?)  
widget para (tamanho da janela, threshold)
exemplo em canto_imagem - mostrar falhas e mapa

### Harris - Matriz Hessiana, auto valores

fórmula  
explicar os elementos da fórmula (?)   
widget para (tamanho da janela, threshold)  
exemplo em canto_imagem - mostrar falhas e mapa

### FAST

método  
exemplo grayscale (?)   
widget para (tamanho da janela, threshold)   
exemplo em canto_imagem - mostrar falhas e mapa